# How do I _copy_ a file from the _Public Reference_ files?
### Overview
Files can be copied from other projects you are a member of or public reference. Here we focus on copying a file from _public reference_.

### Prerequisites
 1. You need to be a member (or owner) of _at least one_ project.
 2. You need your _authentication token_ and the API needs to know about it. See <a href="Setup_API_environment.ipynb">**Setup_API_environment.ipynb**</a> for details.
 3. You understand how to <a href="projects_listAll.ipynb" target="_blank">list</a> projects you are a member of (we will just use that call directly and pick one here).
 
## Imports
We import the _Api_ class from the official sevenbridges-python bindings below.

In [ ]:
import sevenbridges as sbg

## Initialize the object
The _Api_ object needs to know your **auth\_token** and the correct path. Here we assume you are using the .sbgrc file in your home directory. For other options see <a href="Setup_API_environment.ipynb">Setup_API_environment.ipynb</a>

In [ ]:
# User input: specify platform {cgc, sbg}
prof = 'cgc'

config_config_file = sbg.Config(profile=prof)
api = sbg.Api(config=config_config_file)

## Copy a file from the Public Reference
We will first find our _source\_project_ (the Public Reference Files), then list the files within the source project<sup>1</sup>, and copy a file from **_source\_project_ -> _my\_project_**.

The critical information for this POST is the **file_id**. Note, you are allow to copy the same file as many times as you like. However, duplicates will be automatically have a prefix attached of (\_1\_, \_2\_, etc) depending on how many times you copy the file.

To make these results very obvious, use an empty project as your MY\_PROJECT (e.g. your <a href=projects_makeNew.ipynb> cookbook example project</a>) or change the _name_ in the _data_ dictionary to something like 'Dept of Awesome.copy'

<sup>1</sup> Files are only accessible **within** a project - here the Public Reference project (**warning** we may change this project name in the future).

In [ ]:
# [USER INPUT] Set your project name; source project name; and file (f_) indices here:
source_project_id = 'admin/sbg-public-data'        
my_project_name = 'Keep on Smiling'
f_index = 12                                     # file to copy


# Find your project
my_project = [p for p in api.projects.query(limit=100).all() \
                  if p.name == my_project_name]

# Double-check that your project exists
if not my_project:
    print('Target project (%s) not found, check spelling' % my_project_name)
    raise KeyboardInterrupt
else:
    my_project = my_project[0]

# LIST all files in the source and target project
my_files = api.files.query(limit = 100, project = my_project.id).all()
source_files = list(api.files.query(limit = 100, project = source_project_id).all())

# pop out the file names 
my_file_names = [f.name for f in my_files]
source_file_names = [f.name for f in source_files]

# Check if first file already exists in the target project
if source_file_names[f_index] in my_file_names:
    print('file already exists in second project, please try another file')
else:
    print('File (%s) does not exist in Project (%s); copying now' % \
          (source_file_names[f_index], my_project.id))
    source_file = [curr_file for curr_file in source_files \
                        if curr_file.name == source_file_names[f_index]][0]
    
    my_new_file = source_file.copy(project = my_project.id, \
                                   name = source_file.name)

    # re-list files in target project to verify the copy worked
    my_files = [f.name for f in api.files.query(limit = 100, project = my_project.id).all()]
    
    if source_file.name in my_files:
        print('Sucessfully copied one file!')
    else:
        print('Something went wrong...')

## Additional Information
Detailed documentation of this particular REST architectural style request is available [here](http://docs.cancergenomicscloud.org/docs/copy-a-file)